In [147]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from IPython.display import display

In [149]:
driver = webdriver.Chrome()

In [150]:
api_key = "RGAPI-ec132e4d-77ce-4de1-bb45-fdaf011ba5b6"

In [151]:
leagueacc_url="https://porofessor.gg/live/na/rham57-NA1"

In [152]:
driver.get(leagueacc_url)

In [153]:
ally_tag=list(player.find_element(By.XPATH, '..').get_attribute('data-summonername') for player in driver.find_elements(By.XPATH, "//div[contains(@class, 'card card-5')]/div[contains(@class, 'cardHeader blue')]"))

In [154]:
enemy_tag=list(player.find_element(By.XPATH, '..').get_attribute('data-summonername') for player in driver.find_elements(By.XPATH, "//div[contains(@class, 'card card-5')]/div[contains(@class, 'cardHeader red')]"))

In [155]:
playerTag="rham57#NA1"
playerName, playerTag = playerTag.split("#")
get_puuid_url=f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{playerName}/{playerTag}" + '?api_key=' + api_key
get_puuid_raw=requests.get(get_puuid_url)
while get_puuid_raw.status_code == 429:
        print("Sleeping")
        time.sleep(10)
        get_puuid_raw = requests.get(get_puuid_url)
get_puuid = get_puuid_raw.json()['puuid']
get_current_match_url = f"https://na1.api.riotgames.com/lol/spectator/v5/active-games/by-summoner/{get_puuid}" + '?api_key=' + api_key
get_current_match_raw = requests.get(get_current_match_url)
while get_current_match_raw.status_code == 429:
        print("Sleeping")
        time.sleep(10)
        get_current_match_raw = requests.get(get_current_match_url)
get_current_match_puuids = get_current_match_raw.json()['participants']

all_matches = []

In [156]:
for playerNumbers in range(10):
    print(get_current_match_puuids[playerNumbers].get('riotId'))
    player_puuid=get_current_match_puuids[playerNumbers].get('puuid')
    matches_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{player_puuid}/ids?start=0&count=30" + '&api_key=' + api_key
    matches_info_raw = requests.get(matches_api_url)
    while matches_info_raw.status_code == 429:
        print("Sleeping")
        time.sleep(10)
        matches_info_raw = requests.get(matches_api_url)
    matches_info= list(matches_info_raw.json())
        
    for match in matches_info:
        match_api_url= f"https://americas.api.riotgames.com/lol/match/v5/matches/{match}" + '?api_key=' + api_key
        match_info_raw = requests.get(match_api_url)
        while match_info_raw.status_code == 429:
            print("Sleeping")
            time.sleep(10)
            match_info_raw = requests.get(match_api_url)
        match_info= match_info_raw.json()
        participantIndex = match_info['metadata']['participants'].index(player_puuid)
        matchData=match_info['info']['participants'][participantIndex]
        if (match_info['info']['queueId']==420) and (matchData['timePlayed']>180):
            solo_testing_key_stats = ['championName','teamPosition','kills', 'deaths', 'assists', 'damageDealtToBuildings', 'deaths', 'visionScore','doubleKills', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'goldEarned', 'inhibitorKills', 'inhibitorTakedowns', 'killingSprees', 'objectivesStolen', 'totalMinionsKilled', 'totalDamageDealt', 'turretKills','win']
            team_testing_key_stats = ['kda', 'damagePerMinute', 'goldPerMinute', 'teamDamagePercentage','dodgeSkillShotsSmallWindow', 'takedowns']
            support_solo_key_stats = ['detectorWardsPlaced']
            support_team_key_stats = ['controlWardsPlaced', 'completeSupportQuestInTime']
            jungle_solo_key_stats = ['neutralMinionsKilled', 'detectorWardsPlaced', 'dragonKills']
            jungle_team_key_stats = ['jungleCsBefore10Minutes']
            base_stats = {
                    **{stat: matchData[stat] for stat in solo_testing_key_stats},
                    **{stat: matchData['challenges'][stat] for stat in team_testing_key_stats}
                }
            if matchData['teamPosition']== 'UTILITY':
                role_stats={
                    **{stat: matchData[stat] for stat in support_solo_key_stats},
                    **{stat: matchData['challenges'][stat] for stat in support_team_key_stats}
                }
            elif matchData['teamPosition']== 'JUNGLE':
               role_stats={
                    **{stat: matchData[stat] for stat in jungle_solo_key_stats},
                    **{stat: matchData['challenges'][stat] for stat in jungle_team_key_stats}
                }
            else:
                role_stats={}
            timePlayed = f"{matchData['timePlayed'] // 60}:{matchData['timePlayed'] % 60:02d}"
            all_matches.append({'player': get_current_match_puuids[playerNumbers].get('riotId'), 'allyOrEnemy': get_current_match_puuids[playerNumbers].get('teamId'), 'timePlayed' : timePlayed, **base_stats, **role_stats})

aryk#003
Good Regular #NA1
Nnuyanuel#NA1
ALLMYFAULT#SAD1
yeaokwhatever#NA1
rham57#NA1
NamiToes#uwu
Sleeping
Sleeping
Sleeping
Sleeping
Llienalsay#NA1
Greennnnn#NA1
Jawdrop#NA1
Sleeping
Sleeping
Sleeping
Sleeping


In [167]:
df=pd.DataFrame(all_matches)
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_row', None)  
pd.set_option('display.expand_frame_repr', False)
display(df)

,player,allyOrEnemy,timePlayed,championName,teamPosition,kills,deaths,assists,damageDealtToBuildings,visionScore,doubleKills,firstBloodAssist,firstBloodKill,firstTowerAssist,firstTowerKill,goldEarned,inhibitorKills,inhibitorTakedowns,killingSprees,objectivesStolen,totalMinionsKilled,totalDamageDealt,turretKills,win,kda,damagePerMinute,goldPerMinute,teamDamagePercentage,dodgeSkillShotsSmallWindow,takedowns,detectorWardsPlaced,controlWardsPlaced,completeSupportQuestInTime,neutralMinionsKilled,dragonKills,jungleCsBefore10Minutes
0,aryk#003,100,32:08,Janna,UTILITY,5,3,22,1206,69,1,False,False,False,False,10224,0,2,1,0,25,28488,0,True,9.000000,315.611910,318.092106,0.077617,0,27,4.0,4.0,0.0,NaN,NaN,NaN
1,aryk#003,100,16:25,Nami,UTILITY,1,0,7,771,21,0,False,True,True,False,4674,0,0,0,0,15,10549,0,True,8.000000,276.461948,284.626368,0.106639,0,8,1.0,1.0,1.0,NaN,NaN,NaN
2,aryk#003,100,26:05,Nami,UTILITY,0,6,13,209,46,0,False,False,False,False,6136,0,0,0,0,17,14317,0,False,2.166667,261.303979,235.160560,0.080154,4,13,4.0,4.0,0.0,NaN,NaN,NaN
3,aryk#003,100,32:30,Soraka,UTILITY,1,4,18,501,81,0,False,False,False,False,8408,0,0,0,0,35,21934,0,False,4.750000,326.727728,258.703635,0.080732,1,19,5.0,5.0,1.0,NaN,NaN,NaN
4,aryk#003,100,35:33,Nami,UTILITY,1,5,3,1342,70,0,False,False,True,False,7998,0,0,0,0,24,21289,0,False,0.800000,144.580905,224.955054,0.065054,0,4,5.0,5.0,1.0,NaN,NaN,NaN
5,aryk#003,100,31:05,Janna,UTILITY,2,7,21,98,46,0,False,False,False,False,8605,0,0,0,0,19,21775,0,False,3.285714,366.637212,276.749474,0.092644,2,23,5.0,5.0,1.0,NaN,NaN,NaN
6,aryk#003,100,33:07,Lulu,UTILITY,2,10,15,968,60,0,False,False,True,False,8252,0,0,0,0,24,25416,0,False,1.700000,198.884373,249.077716,0.062020,0,17,5.0,5.0,1.0,NaN,NaN,NaN
7,aryk#003,100,35:38,Seraphine,UTILITY,4,8,22,1035,85,0,False,True,False,False,10738,0,1,0,0,36,50940,1,True,3.250000,629.841044,301.207750,0.116497,2,26,5.0,5.0,0.0,NaN,NaN,NaN
8,aryk#003,100,35:02,Lulu,UTILITY,2,2,26,301,88,0,False,False,False,False,10815,2,4,0,0,28,34848,0,True,14.000000,300.644006,308.661419,0.067939,0,28,8.0,8.0,1.0,NaN,NaN,NaN
9,aryk#003,100,28:29,Lulu,UTILITY,2,4,26,483,54,0,True,False,False,False,8256,0,0,0,0,24,25731,0,True,7.000000,302.056849,289.821144,0.069364,13,28,7.0,7.0,1.0,NaN,NaN,NaN


In [132]:
requests.get(api)

NameError: name 'api' is not defined

In [ ]:
# for player in ally_tag:
#     print(player)
#     name, tag = player.split("#")
#     puuid_api_url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{name}/{tag}" + '?api_key=' + api_key
#     # player_info_raw = requests.get(puuid_api_url)
#     player_info = requests.get(puuid_api_url).json()['puuid']
#     matches_api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{player_info}/ids?start=0&count=20" + '&api_key=' + api_key
#     # matches_info_raw = requests.get(matches_api_url)
#     matches_info = list(requests.get(matches_api_url).json())
#     for match in matches_info:
#         match_api_url= f"https://americas.api.riotgames.com/lol/match/v5/matches/{match}" + '?api_key=' + api_key
#         # match_info_raw = requests.get(match_api_url)
#         match_info =requests.get(match_api_url).json()
#         participantIndex = match_info['metadata']['participants'].index(player_info)
#         print(match_info['info']['participants'][participantIndex]['kills'], end =" ")
#         print(match_info['info']['participants'][participantIndex]['deaths'], end =" ")
#         print(match_info['info']['participants'][participantIndex]['assists'])

In [ ]:
for player in ally_url:
    print(player)

In [429]:
# for player in ally_tag:
#     name, tag = player.split("#")
#     name = name.replace(" ", "%20")
#     tag = tag.replace(" ", "%20")
#     ally_url.append(f"https://www.op.gg/summoners/na/{name}-{tag}")
#     print(f"https://www.op.gg/summoners/na/{name}-{tag}")
# for player in enemy_tag:
#     name, tag = player.split("#")
#     name = name.replace(" ", "%20")
#     tag = tag.replace(" ", "%20")
#     ally_url.append(f"https://www.op.gg/summoners/na/{name}-{tag}")
#     print(f"https://www.op.gg/summoners/na/{name}-{tag}")
# -----------------------------------------------------------------------------------------------------------------------------------------------------------------
# for player in ally_url:
#     driver.get(player)
#     driver.get(ally_url[0])
#     kills=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='kills']").text)
#     deaths=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='deaths']").text)
#     assists=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='assists']").text)
#     kda=(kills+assists)/deaths
#     ally_kda_list.append(kda)
#     print(kda)kills=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='kills']").text)
# deaths=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='deaths']").text)
# assists=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='assists']").text)
# kda=(kills+assists)/deaths
# ally_kda_list.append(kda)
# print(kda)
# for player in enemy_url:
#     driver.get(player)
#     kills=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='kills']").text)
#     deaths=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='deaths']").text)
#     assists=float(driver.find_element(By.XPATH, "//div[@data-tab-id='championsData-soloqueue' and @class= 'content active']/div[@class='number-only-chart text-center']/div[@class='number']/span[@class='assists']").text)
#     kda=(kills+assists)/deaths
#     enemy_kda_list.append(kda)
#     print(kda)
# ally_average_kda=sum(ally_kda_list)/5
# enemy_average_kda=sum(enemy_kda_list)/5